# Question 1

In this question, we will solve the 3-coloring problem using melcular computing.

Given a graph $G = (V, E)$, the 3-coloring problem is to assign one of the three colors to each vertex in $V$ such that no two adjacent vertices have the same color.

Let's define a graph $G'$ as follows:
$$
t_1 \rightarrow (t_{1_A}, t_{1_B}, t_{1_C}) \rightarrow t_2 \rightarrow ... \rightarrow (t_{|V|_A}, t_{|V|_B}, t_{|V|_C}) \rightarrow z
$$

where $t_i$ is the $i$-th vertex in $V$, $t_{i_A}$, $t_{i_B}$, and $t_{i_C}$ are the colors of the $i$-th vertex, and $z$ is just and end point. Each path from $t_1$ to $z$ in the graph where for each $i$, only one of the edges $(t_i, t_{i_A})$, $(t_i, t_{i_B})$, and $(t_i, t_{i_C})$ is selected is a 3-coloring of the graph.

The path represents a **valid** 3-coloring iff for all adjacent vertices $t_i, t_j$ in $V$, there is no edge $(t_i, t_{i_X})$ and $(t_j, t_{j_X})$ selected for the same color $X$.

We will build all DNA strands that represent a path in $G'$ and then filter out the invalid paths. If we have any remaining paths, then the graph $G$ is 3-colorable.

1. For each vertice $t$, in $G'$, we will build a unique sequence composing of 20 bases (or more if $G'$ is too big), and build a sequence for the edges in $G'$ like Adleman did in his paper which was taught in the lecture.

2. Amplify the DNA strands that represent a path from $t_1$ to $z$ in $G'$ with PCR using the primers for $t_1$ and $z$.

3. Run the DNA strands through a gel electrophoresis to separate the strands by length, and keep only those strands that we amplified in step 2, with length $(2 \cdot |V| + 1) \cdot 20$, leaving us with only the paths from $t_1$ to $z$.

4. Filter out the invalid paths by checking if there are any adjacent vertices $t_i, t_j$ in $V$ such that there is an edge $(t_i, t_{i_X})$ and $(t_j, t_{j_X})$ selected for the same color $X$. We will do this using an iterative process for all edges $(t_i, t_j)$ in $E$ and all colors $X$:
    - We will take all the DNA strands with the subsequence represented by $(t_i, t_{i_X})$ to another tube - $T_1$ using magnetic beads.
    - From $T_1$, we will take all the DNA strands with the subsequence represented by $(t_j, t_{j_X})$ to another tube - $T_2$ using magnetic beads.
    - We drop the contents of $T_2$, and keep the contents of $T_1$ in the original tube.

5. If there are any DNA strands left in the tube, then the graph $G$ is 3-colorable.

# Question 2

This notebook demonstrates a virtual “molecular computing” lab that uses DNA strands as the data structure
and implements various “wet lab” processes such as synthesis, separation, hybridization, etc.

We will also implement a simple 3-SAT solver that uses these molecular computing operations,
following the methods taught in the chapter.

## Imports & Setup

In [1]:
import numpy as np
import random
import unittest
from typing import List, Tuple, Union, Optional

## DNAStrand Class

In [2]:
class DNAStrand:
    """
    A class representing a single or double-stranded DNA.

    Attributes:
    -----------
    sequence : str
        The nucleotide sequence composed of A, C, G, T (for single strands).
    complement_sequence : str or None
        The complementary strand if present (otherwise None).
        If complement_sequence is not None, the strand is effectively double-stranded.

    Methods:
    --------
    check_double_helix(other_strand):
        Checks if this strand forms a stable double helix with `other_strand`.
        Returns True if they can form a fully matching complementary sequence.

    is_double_stranded():
        Returns True if the strand is currently double-stranded (complement_sequence is not None).
    """

    # Define base complements
    base_complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}

    def __init__(self, sequence: str, complement_sequence: Optional[str] = None) -> None:
        self.sequence = sequence.upper()
        self.complement_sequence = complement_sequence.upper() if complement_sequence else None

    def __repr__(self) -> str:
        if self.is_double_stranded():
            return f"DNAStrand({self.sequence}|{self.complement_sequence})"
        else:
            return f"DNAStrand({self.sequence})"
        
    def __len__(self) -> int:
        return len(self.sequence)

    def is_double_stranded(self) -> bool:
        return self.complement_sequence is not None

    def check_double_helix(self, other_strand: 'DNAStrand') -> bool:
        """
        Checks if this strand can form a stable double helix with `other_strand`.
        We assume that a double helix is formed if the sequences are complementary
        and of the same length.
        """
        if len(self.sequence) != len(other_strand.sequence):
            return False
        # Check complementarity
        for s1, s2 in zip(self.sequence, other_strand.sequence):
            if self.base_complement[s1] != s2:
                return False
        return True

    def get_complement(self) -> str:
        """Return the complement of the current single-stranded sequence."""
        return ''.join([self.base_complement[base] for base in self.sequence])

## DNA Test Tube Class

In [3]:
class DNATestTube:
    """
    A class representing a test tube containing one or more DNAStrand objects.

    Attributes:
    -----------
    strands : List[DNAStrand]
        The list of DNA strands currently in the test tube.

    Probabilistic parameters:
    -------------------------
    p_amplification_fail : float
        Probability that a PCR amplification of a strand fails even if it exists.
    p_hybridize_fail : float
        Probability that a hybridization reaction fails even if complement strands exist.
    """

    def __init__(self, strands=None, p_amplification_fail=0.05, p_hybridize_fail=0.05) -> None:
        if strands is None:
            strands = []
        self.strands: List[DNAStrand] = strands
        self.p_amplification_fail = p_amplification_fail
        self.p_hybridize_fail = p_hybridize_fail

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.strands})"

    # ------------------
    #   Lab Processes
    # ------------------

    def synthesize(self, sequence: str) -> None:
        """
        Simulate the synthesis of new single-stranded DNA and place it in the tube.
        """
        new_strand = DNAStrand(sequence)
        self.strands.append(new_strand)

    def unify(self, other_tube: 'DNATestTube') -> None:
        """
        Merge the contents of another DNA test tube into this one.
        """
        self.strands.extend(other_tube.strands)
        other_tube.strands.clear()  # The other tube becomes empty.

    def separate(self) -> None:
        """
        Separate double-stranded DNA into single strands (denature them),
        storing each single strand as a separate DNAStrand. This simulates
        a high-temperature denaturation step.
        """
        separated_strands = []
        for strand in self.strands:
            if strand.is_double_stranded():
                # Separate into two single strands
                s1 = DNAStrand(strand.sequence)
                s2 = DNAStrand(strand.complement_sequence)
                separated_strands.extend([s1, s2])
            else:
                separated_strands.append(strand)
        self.strands = separated_strands

    def hybridize(self) -> None:
        """
        Allow single strands that are complementary to bind and form double strands.
        This simulates an annealing step, and we do it probabilistically (some fraction fails).
        We attempt to pair up strands in a naive manner (in pairs).
        """
        # We'll create a dictionary by length of strands for faster grouping
        length_dict = {}
        for strand in self.strands:
            length_dict.setdefault(len(strand.sequence), []).append(strand)
        
        new_strands = []
        used = set()  # Keep track of strands that have already formed a double helix

        for length, group in length_dict.items():
            # Attempt to pair them up
            group_size = len(group)
            checked_indices = set()

            for i in range(group_size):
                if i in checked_indices or i in used:
                    continue
                s1 = group[i]
                if np.random.rand() < self.p_hybridize_fail:
                    # Reaction fails for s1. We won't try to pair it up now
                    new_strands.append(s1)
                    checked_indices.add(i)
                    continue

                pair_found = False
                # Try to find a complementary partner s2
                for j in range(i+1, group_size):
                    if j in checked_indices or j in used:
                        continue
                    s2 = group[j]
                    # Attempt pairing
                    if s1.check_double_helix(s2):
                        # Form a double helix
                        ds = DNAStrand(s1.sequence, s2.sequence)
                        new_strands.append(ds)
                        # Mark them used
                        checked_indices.update([i, j])
                        used.update([i, j])
                        pair_found = True
                        break
                if not pair_found and i not in checked_indices:
                    # No complementary partner found or reaction failed
                    new_strands.append(s1)
                    checked_indices.add(i)

        self.strands = new_strands

    def length_sort(self) -> None:
        """
        Sort strands based on their length (shortest first), purely for demonstration.
        In a wet-lab, this is akin to running them on a gel electrophoresis and grouping
        them by length. Here, we just reorder them internally.
        """
        self.strands.sort(key=len)

    def sequence_strands(self) -> List[str]:
        """
        Simulate reading (sequencing) all single-stranded sequences in the test tube.
        If double-stranded, read only the 'main' sequence (or both, if desired).
        """
        sequences = []
        for strand in self.strands:
            if strand.is_double_stranded():
                # We might choose to read both, but let's just read the "top" for demonstration
                sequences.append(strand.sequence)
            else:
                sequences.append(strand.sequence)
        return sequences

    def amplify(self, target_sequence: str) -> None:
        """
        Amplify (PCR) the given target_sequence if it exists in the test tube.
        Introduces new copies of any strand that matches the target sequence (on either strand
        if double-stranded). Probabilistic failures may apply.
        """
        new_strands = []
        for strand in self.strands:
            match_single = (not strand.is_double_stranded() and strand.sequence == target_sequence)
            match_double = (
                strand.is_double_stranded() and
                (strand.sequence == target_sequence or strand.complement_sequence == target_sequence)
            )
            if match_single or match_double:
                if np.random.rand() > self.p_amplification_fail:
                    # Reaction succeeds – we make a duplicate
                    # In a real scenario, PCR would double every cycle,
                    # but for simplicity, we just add a certain number of copies (say 2 extra).
                    new_strands.append(strand)
                    new_strands.append(strand)
        self.strands.extend(new_strands)

    def extract(self, target_sequence: str) -> None:
        """
        Extract (keep) only those strands that contain the target_sequence (on either 
        strand if double-stranded). 
        Discard others. This is similar to using a magnetic probe that binds only to a 
        certain sequence and then capturing everything that is bound.
        """
        filtered = []
        for strand in self.strands:
            if strand.is_double_stranded():
                if target_sequence in strand.sequence or target_sequence in strand.complement_sequence:
                    filtered.append(strand)
            else:
                if target_sequence in strand.sequence:
                    filtered.append(strand)
        self.strands = filtered

    def cleave(self, cut_sequence: str) -> None:
        """
        Cleave (cut) the DNA at specific restriction sites matching `cut_sequence`.
        Resulting fragments remain in the test tube. If double-stranded,
        cut both top and bottom strands accordingly (this is a simplified approach).
        """
        complementary_cut = self._get_complement(cut_sequence)
        new_strands = []

        for strand in self.strands:
            if strand.is_double_stranded():
                # Cleave both top and bottom (naive approach).
                # We'll just cut the top and treat the result as separate double-stranded fragments if possible.
                fragments_top = self._cleave_single_strand(strand.sequence, cut_sequence)
                fragments_bot = self._cleave_single_strand(strand.complement_sequence, complementary_cut)
                # Attempt to pair them up again by length
                # For simplicity, we pair up by matching indices from the split
                for ft, fb in zip(fragments_top, fragments_bot):
                    new_strands.append(DNAStrand(ft, fb))
                # If mismatch in #fragments, keep leftover as single-stranded (rare corner case in naive approach)
                if len(fragments_top) > len(fragments_bot):
                    for ft in fragments_top[len(fragments_bot):]:
                        new_strands.append(DNAStrand(ft))
                elif len(fragments_bot) > len(fragments_top):
                    for fb in fragments_bot[len(fragments_top):]:
                        new_strands.append(DNAStrand(fb))
            else:
                # Single-stranded cleavage
                fragments = self._cleave_single_strand(strand.sequence, cut_sequence)
                for fragment in fragments:
                    new_strands.append(DNAStrand(fragment))

        self.strands = new_strands

    def _cleave_single_strand(self, sequence: str, cut_sequence: str) -> List[str]:
        """
        Cut the single-stranded DNA into fragments by removing the cut_sequence as a delimiter.
        """
        return sequence.split(cut_sequence)
    
    def _get_complement(self, seq: str) -> str:
        """
        Return the *forward* complement of seq (not reversed).
        e.g. 'AC' -> 'TG'
        """
        comp = []
        for b in seq:
            comp.append(DNAStrand.base_complement[b])
        return ''.join(comp)

In [ ]:
class TestDNATestTube(unittest.TestCase):

    def setUp(self):
        np.random.seed(0)
        random.seed(0)

    def test_init(self):
        tube = DNATestTube()
        self.assertEqual(len(tube.strands), 0, "Initial DNATestTube should be empty.")

        s1 = DNAStrand("ACGT")
        s2 = DNAStrand("TTTT")
        tube2 = DNATestTube(strands=[s1, s2])
        self.assertEqual(len(tube2.strands), 2, "Should have 2 initial strands.")

    def test_synthesize(self):
        tube = DNATestTube()
        tube.synthesize("ACGTACGT")
        self.assertEqual(len(tube.strands), 1)
        self.assertEqual(tube.strands[0].sequence, "ACGTACGT")

    def test_unify(self):
        tube1 = DNATestTube()
        tube1.synthesize("AAAA")
        tube2 = DNATestTube()
        tube2.synthesize("CCCC")

        tube1.unify(tube2)
        self.assertEqual(len(tube1.strands), 2)
        self.assertEqual(len(tube2.strands), 0)

    def test_separate(self):
        ds = DNAStrand("ACGT", "TGCA")
        tube = DNATestTube([ds])
        tube.separate()
        self.assertEqual(len(tube.strands), 2, "Should have separated into two single strands.")
        self.assertFalse(tube.strands[0].is_double_stranded())

    def test_hybridize(self):
        s1 = DNAStrand("ACGT")
        s2 = DNAStrand("TGCA")
        tube = DNATestTube([s1, s2], p_hybridize_fail=0.0)
        tube.hybridize()
        self.assertEqual(len(tube.strands), 1, "Should have formed one double strand.")
        self.assertTrue(tube.strands[0].is_double_stranded())

    def test_length_sort(self):
        s1 = DNAStrand("ACG")
        s2 = DNAStrand("ACGTACGT")
        s3 = DNAStrand("ACGT")
        tube = DNATestTube([s2, s1, s3])
        tube.length_sort()
        lengths = [len(s.sequence) for s in tube.strands]
        self.assertEqual(lengths, [3, 4, 8], "Strands sorted by length ascending.")

    def test_sequence_strands(self):
        s1 = DNAStrand("ACGT")
        s2 = DNAStrand("AAAA", "TTTT")
        tube = DNATestTube([s1, s2])
        seqs = tube.sequence_strands()
        self.assertIn("ACGT", seqs)
        self.assertIn("AAAA", seqs)
        self.assertNotIn("TTTT", seqs)

    def test_amplify(self):
        s1 = DNAStrand("ACGT")
        s2 = DNAStrand("AAAA")
        s3 = DNAStrand("ACGT")
        tube = DNATestTube([s1, s2, s3], p_amplification_fail=0.0)
        tube.amplify("ACGT")
        count_acgt = sum(1 for s in tube.strands if s.sequence == "ACGT")
        self.assertEqual(count_acgt, 6, "With no PCR failure, each matching strand should have been tripled.")

    def test_extract(self):
        s1 = DNAStrand("ACGT")
        s2 = DNAStrand("GGGG")
        s3 = DNAStrand("ACCA", "TGGT")
        tube = DNATestTube([s1, s2, s3])
        tube.extract("AC")
        self.assertEqual(len(tube.strands), 2)
        self.assertIn(s1, tube.strands)
        self.assertIn(s3, tube.strands)

    def test_cleave(self):
        s1 = DNAStrand("ACGTTACG")
        s2 = DNAStrand("ACGTACGT", "TGCATGCA")
        tube = DNATestTube([s1, s2])
        tube.cleave("AC")

        all_seq = [st.sequence for st in tube.strands if not st.is_double_stranded()]
        ds_seq_pairs = [(st.sequence, st.complement_sequence)
                        for st in tube.strands if st.is_double_stranded()]

        # Check single-stranded fragments from s1
        self.assertIn("", all_seq, "Empty fragment from leading 'AC' split.")
        self.assertIn("GTT", all_seq)
        self.assertIn("G", all_seq)

        # The test expects:
        #   top => ["", "GT", "GT"]
        #   bottom => ["", "CA", "CA"]
        # so we need at least one ds fragment matching one of these pairs
        expected_top = ["", "GT", "GT"]
        expected_bottom = ["", "CA", "CA"]
        any_match = False
        for (top, bot) in ds_seq_pairs:
            if top in expected_top and bot in expected_bottom:
                any_match = True
                break
        self.assertTrue(any_match, 
                        "At least one double-stranded fragment should match top & bottom from splits.")

# Run the tests
if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

## Basic Lab Operations

The following code block contains basic testing and demonstration of the implemented lab operations.

In [ ]:
print("=== DEMO: Basic Lab Operations ===")
# Create a test tube
tube = DNATestTube()

# 1. Synthesize
tube.synthesize("ACGTACGT")
tube.synthesize("TGCA")  # complementary to ACGT but shorter
tube.synthesize("ACGT")  # complementary to TGCA
print("After Synthesis:", tube)

# 2. Hybridize
tube.separate()  # ensure they're single first (not strictly needed if newly synthesized)
tube.hybridize()
print("After Hybridize:", tube)

# 3. Length Sort (like running on a gel)
tube.length_sort()
print("After Length Sort:", tube)

# 4. Amplify
# Try amplifying "ACGTACGT"
tube.amplify("ACGTACGT")
print("After Amplify (ACGTACGT):", tube)

# 5. Extract
# Keep only strands that contain "ACGT" on either top or bottom
tube.extract("ACGT")
print("After Extract (ACGT):", tube)

# 6. Cleave
# Cut at "AC"
tube.cleave("AC")
print("After Cleave ('AC'):", tube)

# 7. Sequence
print("Final Sequencing Results:", tube.sequence_strands())
print("================================\n")

## 3-SAT Solver

In this section, we will implement a simple 3-SAT solver that uses the molecular computing operations we have implemented.

In [6]:
def three_sat_molecular_solver(clauses: List[Tuple[int, int, int]],
                               num_vars: int,
                               p_amplification_fail=0.05,
                               p_hybridize_fail=0.05) -> bool:
    """
    A (very) simplified demonstration of the 3-SAT solver using molecular computing steps.
    Each assignment to the n variables is encoded as an n-length string of A/T representing True/False
    (or vice versa). We generate 2^n possible assignments and put them in one test tube.

    Then, for each clause, we extract (or eliminate) those assignments that do not satisfy that clause.
    If, after processing all clauses, there is at least one assignment left, the formula is satisfiable.

    Parameters:
    -----------
    clauses : list of tuples (x, y, z)
        Each element x,y,z is either an integer representing the variable index
        (e.g. +1 or -1 for True/False assignment), or None for demonstration (not used).
        For example, (1, -2, 3) means (x1 OR NOT x2 OR x3).
    num_vars : int
        Number of boolean variables in the formula.

    p_amplification_fail : float
        Probability that a PCR amplification fails.
    p_hybridize_fail : float
        Probability that a hybridization fails.

    Returns:
    --------
    satisfiable : bool
        True if the formula is satisfiable, False otherwise.
    """

    # Create a test tube with all possible assignments (2^num_vars)
    # We'll encode variable i = A for True, T for False (just for example).
    all_assignments = []
    for assignment_value in range(2**num_vars):
        # e.g. assignment_value = 0 -> all false, 1 -> 0001 in binary, etc.
        # Convert to a binary string
        bin_str = bin(assignment_value)[2:].zfill(num_vars)
        # Map each bit: '0'-> 'T', '1'-> 'A' (arbitrary choice)
        dna_seq = ''.join('A' if b == '1' else 'T' for b in bin_str)
        all_assignments.append(dna_seq)

    # Put them in the test tube
    tube = DNATestTube(
        [DNAStrand(seq) for seq in all_assignments],
        p_amplification_fail,
        p_hybridize_fail
    )
    print("Initial test tube assignments:")
    print(tube.sequence_strands())

    # For each clause, eliminate (extract) only the assignments that satisfy that clause.
    # A clause (x, y, z) is satisfied by an assignment if at least one literal is True.
    # literal x>0 => variable x is True, x<0 => variable abs(x) is False
    # We implement this in a purely string-based manner:
    #   - If variable i is True => the i-th position in the assignment is 'A'
    #   - If variable i is False => the i-th position in the assignment is 'T'
    # We'll keep only the assignments that satisfy the clause.

    for idx, clause in enumerate(clauses):
        # 1) separate => ensure single strands
        tube.separate()

        # We'll define a function that returns True if a single assignment string satisfies the clause
        def satisfies_clause(assignment_str: str, clause: Tuple[int,int,int]) -> bool:
            # For each literal in the clause
            for literal in clause:
                if literal == 0 or literal is None:
                    # ignore placeholder
                    continue
                var_index = abs(literal) - 1  # zero-based index
                # We said: 'A' means True, 'T' means False
                # literal>0 => expecting 'A' at that position
                # literal<0 => expecting 'T' at that position
                if literal > 0 and assignment_str[var_index] == 'A':
                    return True
                if literal < 0 and assignment_str[var_index] == 'T':
                    return True
            return False

        # 2) For the current clause, keep only those that satisfy it
        #    We can do that by building the set of sequences that satisfy the clause
        valid_sequences = []
        for s in tube.strands:
            # read top strand only if double-stranded
            seq = s.sequence
            if satisfies_clause(seq, clause):
                valid_sequences.append(seq)

        # now we "extract" these valid sequences:
        # Implementation detail: we re-build the tube with only those that are valid
        new_strands = []
        for s in tube.strands:
            seq = s.sequence
            if seq in valid_sequences:
                new_strands.append(s)
        tube.strands = new_strands

        print(f"\nAfter processing clause {idx+1} {clause}, remaining assignments:")
        print(tube.sequence_strands())

        # If the tube is empty, no assignments left => unsatisfiable
        if not tube.strands:
            print("No assignments satisfy the formula. UNSATISFIABLE.\n")
            return False

    # If we still have strands left, at least one assignment satisfies all clauses
    print("\nAt least one assignment remains. SATISFIABLE.\n")
    return True

In [ ]:
# 3-SAT solving demonstration
# Suppose we have a formula in 3-SAT with 3 variables and 2 clauses:
# (x1 OR NOT x2 OR x3) AND (NOT x1 OR x2 OR x3)
# We encode this as: (1, -2, 3), (-1, 2, 3)
# x1 => variable #1, -2 => NOT variable #2, x3 => variable #3
# For simplicity, let's also allow "0" or None placeholders if needed.

# First example: 3 variables, 4 clauses - SATISFIABLE
clauses_example = [(1, -2, 3), (-1, 2, 3), (1, 1, 1), (-1, -1, -1)]
num_vars_example = 3

satisfiable = three_sat_molecular_solver(clauses_example, num_vars_example)
print(f"Final result of 3-SAT solver: {satisfiable=}")

In [ ]:
# Second example: 2 variables, 2 clauses - SATISFIABLE
clauses_example_2 = [(1, 2, 1), (-1, 2, 1)]
num_vars_example_2 = 2

satisfiable = three_sat_molecular_solver(clauses_example_2, num_vars_example_2)
print(f"Final result of 3-SAT solver: {satisfiable=}")

In [ ]:
# Third example: 4 variables, 5 clauses - UNSATISFIABLE
clauses_example_3 = [(1, 2, 2), (3, 2, 4), (-4, 3, 3), (4, 4, 4), (-3, -3, -4)]
num_vars_example_3 = 4

satisfiable = three_sat_molecular_solver(clauses_example_3, num_vars_example_3)
print(f"Final result of 3-SAT solver: {satisfiable=}")

## Discussion

In a “wet lab,” these processes (synthesis, separation, hybridization, etc.) are carried out
using actual biochemical reactions. The DNA strands are physically manipulated, separated
in a gel, amplified by PCR, and so on. Each step has a real-world limitation in terms of
reagent concentrations, reaction efficiencies, handling errors, and time constraints.

In a “virtual lab,” as demonstrated here in Python, we simulate these operations in silico.
We idealize or simplify many of the realities of molecular biology: for instance, we assume
we can easily identify sequences, or that we do not lose material except for a small
probability of reaction failure. Additionally, the time complexity in a real wet lab does
not necessarily scale in the same way as a computational simulation, because DNA operations
can, in theory, be done massively in parallel, whereas in a standard CPU-based program
we face typical computational constraints of memory and sequential processing steps. But still, the time complexity of the simulation is $O(m \cdot 2^n \cdot n)$ where $m$ is the number of clauses, $n$ is the number of variables, and $2^n$ is the number of possible assignments.

This demonstration highlights how the theoretical ideas of molecular computing can be mapped
into code, but in practice, the real world is more complex, error-prone, and not as easily
controlled or scaled as a Python simulation might suggest.